# Lab 3 Exercise 3
## Binary Classification

We are given a dataset that consists of biological features of a person and whether or not they are a smoker.

There are *22 descriptive features* and *1 target column* in the dataset as described in the Table below. You are tasked with creating a predictive model to predict the column "SMK_stat_type_cd", which specifies smoker/non-smoker status of the person. You are expected to use classification methods that was shown in the previous exercises.



|      Column      |                                 Description                                 |
|:----------------:|:---------------------------------------------------------------------------:|
| identified_gender              | male, female                                                                |
| age              | round up to 5 years                                                         |
| height           | round up to 5 cm[cm]                                                        |
| weight           | [kg]                                                                        |
| sight_left       | eyesight(left)                                                              |
| sight_right      | eyesight(right)                                                             |
| hear_left        | hearing left, 1(normal), 2(abnormal)                                        |
| hear_right       | hearing right, 1(normal), 2(abnormal)                                       |
| SBP              | Systolic blood pressure[mmHg]                                               |
| DBP              | Diastolic blood pressure[mmHg]                                              |
| BLDS             | BLDS or FSG(fasting blood glucose)[mg/dL]                                   |
| tot_chole        | total cholesterol[mg/dL]                                                    |
| HDL_chole        | HDL cholesterol[mg/dL]                                                      |
| LDL_chole        | LDL cholesterol[mg/dL]                                                      |
| triglyceride     | triglyceride[mg/dL]                                                         |
| hemoglobin       | hemoglobin[g/dL]                                                            |
| urine_protein    | protein in urine, 1(-), 2(+/-), 3(+1), 4(+2), 5(+3), 6(+4)                  |
| serum_creatinine | serum(blood) creatinine[mg/dL]                                              |
| SGOT_AST         | SGOT(Glutamate-oxaloacetate transaminase) AST(Aspartate transaminase)[IU/L] |
| SGOT_ALT         | ALT(Alanine transaminase)[IU/L]                                             |
| gamma_GTP        | y-glutamyl transpeptidase[IU/L]                                             |
| SMK_stat_type_cd **(Target)** | Smoking state, 0(never), 1(active smoker)                      |


### The stages of this problem can be decomposed as follows:
1. Data Preparation
* Ensure data is in correct format (numerical and not string)
* Normalize the data for better convergence (optional)
* Split the data into train/test subsets

2. Model Selection
* Instanciate models and fit on training data
* Evaluate model performance on testing data
* Select model with best performance

3. Submit model
* Your model will be evaluated on data that is kept separate from training/testing data
* The predictions from your model will be uploaded to the course server where it will be evaluated

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/MyDrive/4c16-labs/code/lab-03/
!unzip -o data.zip
!echo 'data/*' > .gitignore

Mounted at /content/gdrive
/content/gdrive/MyDrive/4c16-labs/code/lab-03
Archive:  data.zip
  inflating: data/data.csv           
  inflating: data/validation.csv     


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, confusion_matrix, ConfusionMatrixDisplay

In [ ]:
# Data Preparation
# The data is supplied as ".csv" format
# There are 23 columns in csv file, with 22 columns being features and 1 column being the target

# The csv file can be read into as a dataframe using the pandas library
DataFrame = pd.read_csv('data/data.csv')

# Shuffle the data
DataFrame = DataFrame.sample(frac=1)

# Visualize the first 5 rows of the imported dataframe
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
5011,Male,45,175,65,76.0,1.0,1.0,1.0,1.0,124.0,...,66.0,144.0,26.0,15.8,1.0,1.0,25.0,20.0,20.0,0.0
33824,Male,60,165,60,75.0,0.5,0.6,1.0,1.0,123.0,...,60.0,116.0,140.0,14.9,1.0,0.9,21.0,15.0,38.0,1.0
13378,Male,40,170,65,80.0,1.0,1.2,1.0,1.0,133.0,...,55.0,138.0,147.0,16.0,1.0,0.9,27.0,38.0,37.0,0.0
19914,Male,25,160,45,63.5,0.7,0.6,1.0,1.0,118.0,...,41.0,103.0,72.0,13.0,1.0,0.6,19.0,19.0,30.0,0.0
34836,Female,40,155,40,59.0,1.5,1.5,1.0,1.0,93.0,...,84.0,109.0,62.0,13.6,1.0,0.9,22.0,6.0,10.0,0.0
30209,Female,70,155,55,82.0,0.4,0.5,1.0,1.0,112.0,...,40.0,170.0,93.0,14.2,1.0,1.0,27.0,14.0,12.0,0.0


In [ ]:
# The column "identified_gender" contains non-numeric data. This cannot be used as is and needs to be converted to numerical representations
# We can encode the identified gender to a numerical representation by letting "female" == 0, and "male" == 1.
# This can be done manually by using conditional statements on the dataframe, but luckily for us there is a simpler method
# We can use the built in LabelEncoder function of sklearn to do this

LabelEncoder = sklearn.preprocessing.LabelEncoder()
LabelEncoder.fit(DataFrame['identified_gender'])
DataFrame['identified_gender'] = LabelEncoder.transform(DataFrame['identified_gender'])

# Label encoder sorts the input column of data in alphabetical order, and then assigns a numerical value to each unique entry
# This results in 'female' being mapped to 0, 'male' being mapped to 1
display(DataFrame.head(6))

,identified_gender,age,height,weight,waistline,sight_left,sight_right,hear_left,hear_right,SBP,...,HDL_chole,LDL_chole,triglyceride,hemoglobin,urine_protein,serum_creatinine,SGOT_AST,SGOT_ALT,gamma_GTP,SMK_stat_type_cd
5011,1,45,175,65,76.0,1.0,1.0,1.0,1.0,124.0,...,66.0,144.0,26.0,15.8,1.0,1.0,25.0,20.0,20.0,0.0
33824,1,60,165,60,75.0,0.5,0.6,1.0,1.0,123.0,...,60.0,116.0,140.0,14.9,1.0,0.9,21.0,15.0,38.0,1.0
13378,1,40,170,65,80.0,1.0,1.2,1.0,1.0,133.0,...,55.0,138.0,147.0,16.0,1.0,0.9,27.0,38.0,37.0,0.0
19914,1,25,160,45,63.5,0.7,0.6,1.0,1.0,118.0,...,41.0,103.0,72.0,13.0,1.0,0.6,19.0,19.0,30.0,0.0
34836,0,40,155,40,59.0,1.5,1.5,1.0,1.0,93.0,...,84.0,109.0,62.0,13.6,1.0,0.9,22.0,6.0,10.0,0.0
30209,0,70,155,55,82.0,0.4,0.5,1.0,1.0,112.0,...,40.0,170.0,93.0,14.2,1.0,1.0,27.0,14.0,12.0,0.0


In [ ]:
# We should now separate the input features from the target feature and store them as different variables
# We do this by slicing what columns of data we want from the dataframe
# Let's first see the columns available to us by printing DataFrame.columns
print(DataFrame.columns)

Index(['identified_gender', 'age', 'height', 'weight', 'waistline',
       'sight_left', 'sight_right', 'hear_left', 'hear_right', 'SBP', 'DBP',
       'BLDS', 'tot_chole', 'HDL_chole', 'LDL_chole', 'triglyceride',
       'hemoglobin', 'urine_protein', 'serum_creatinine', 'SGOT_AST',
       'SGOT_ALT', 'gamma_GTP', 'SMK_stat_type_cd'],
      dtype='object')


In [ ]:
# The column titled "SMK_stat_type_cd" is our target, and all remaining variables are our input features.
features_columns = DataFrame.columns[:-1]
target_column = DataFrame.columns[-1:]

DataFrame_X = DataFrame[features_columns]   # Selects only Input features
DataFrame_Y = DataFrame[target_column]      # Selects only Target variable

In [ ]:
# Split Data into Train/Test split
# Change the variable "test_frac" to reflect the percentage/fraction of test data in the resulting split
test_frac = 0.15

Train_X, Test_X, Train_Y, Test_Y = train_test_split(DataFrame_X, DataFrame_Y, test_size=test_frac)
print(f"Number of rows for Training:\t{Train_X.shape[0]}\nNumber of rows for Testing:\t{Test_X.shape[0]}")

Number of rows for Training:	42500
Number of rows for Testing:	7500


In [ ]:
# EDIT THIS CELL
# Construct a dictionary of prediction models to compare
# Uncomment the below dictionary and insert as many prediction models as you like.
# You may have used binary classification models in previous exercises
# You may also have to import these modules/libraries to be able to use them
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis


PredictionModels = {
    'K-NN Classificaiton': KNeighborsClassifier(n_neighbors=5),
    'Decision Tree': DecisionTreeClassifier(max_depth=10),
    'AdaBoost': AdaBoostClassifier(n_estimators=50),
    'Random Forest': RandomForestClassifier(n_estimators=100),
    'Naive Bayes': GaussianNB(),
    'Quadratic Discriminant Analysis': QuadraticDiscriminantAnalysis()
}

models = list(PredictionModels.keys())

print("Fitting models, this may take a while")
for _model in models:
    # This loop goes through the models in the variable "PredictionModels"
    # Complete the below code block to fit to the model to training data "Train_X, Train_Y"
    # Peform predictions on the fitted model on the Train set and Test set
    # compute f1 score
    model = PredictionModels[_model] # Get the model
    # Fit the model to training data
    model.fit(Train_X, Train_Y.values.ravel())
    # Predictions on Train Set
    train_predictions = model.predict(Train_X)
    # Predictions on the Test Set
    test_predictions = model.predict(Test_X)
    # Computing f1 Scores
    # Training Set
    f1_score_train = f1_score(Train_Y, train_predictions)
    # Test Set
    f1_score_test = f1_score(Test_Y, test_predictions)
    print(f"Model: {_model}")
    print(f" Training F1 Score: {f1_score_train}")
    print(f" Test F1 Score: {f1_score_test}")

Fitting models, this may take a while
Model: K-NN Classificaiton
 Training F1 Score: 0.7461050201961916
 Test F1 Score: 0.6363802225871191
Model: Decision Tree
 Training F1 Score: 0.7861637060665476
 Test F1 Score: 0.7415022384347537
Model: AdaBoost
 Training F1 Score: 0.7477445394112062
 Test F1 Score: 0.7578626986810957
Model: Random Forest
 Training F1 Score: 1.0
 Test F1 Score: 0.7573665600269406
Model: Naive Bayes
 Training F1 Score: 0.6957956489855781
 Test F1 Score: 0.71
Model: Quadratic Discriminant Analysis
 Training F1 Score: 0.4323783854279876
 Test F1 Score: 0.4548119198827552


In [ ]:
# Select the model with best f1 score and write down the
# corresponding 'key' value from the 'PredictionModels' variable
# Eg if Logistic Regression is chosen then: chosen_model = 'Logistic Regression'

chosen_model = 'AdaBoost'

In [ ]:
# DO NOT EDIT.
# Generate predictions using "chosen_model" and save to file

backend_features = pd.read_csv('data/validation.csv')
backend_features['identified_gender'] = LabelEncoder.transform(backend_features['identified_gender'])
backend_preds = PredictionModels[chosen_model].predict(backend_features)
np.savez_compressed('lab3_ex3_preds', lab3_model=backend_preds)

# Remember to push your changes to the git server for marking!